### Piano

Run this notebook and see what happens. 

Click the game canvas once it shows up to bring it into focus.

* Keyboard image was adapted from [jack-keyboard](http://jack-keyboard.sourceforge.net/).
* The sampled Xylophone is by the [Versilian Community Sample Library](https://vis.versilstudios.com/vcsl.html).

In [1]:
import sys
import os

import numpy as np

In [2]:
sys.path.insert(0, os.path.abspath('./..'))

In [3]:
import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

In [4]:
from jupylet.audio.effects import ConvolutionReverb
from jupylet.audio.device import get_oscilloscope_as_image, set_effects
from jupylet.audio.sample import Sample
from jupylet.audio.sound import note
from jupylet.audio.synth import tb303
from jupylet.audio import use, play, sleep


In [5]:
app = App(width=512, height=420)#, log_level=logging.INFO)

In [6]:
oscilloscope = Sprite(np.zeros((256, 512, 4), 'uint8'), x=256, y=292)

In [7]:
keyboard_layout = Sprite('images/keyboard.png', x=256, y=82, scale=0.5)

In [8]:
state = State(
    
    amp = 1.,
    ms = 50.,
    
    up = False,
    down = False,
    left = False,
    right = False,
)

In [9]:
label0 = Label('amp: %.1f' % state.amp, x=10, y=194)
label1 = Label('span: %.1f ms' % state.ms, x=10, y=174)
label2 = Label('use ← → ↑ ↓ to modify', anchor_x='right', x=app.width - 10, y=174)

In [10]:
keys = app.window.keys

keyboard = {

    keys.Z: note.C,
    keys.S: note.Cs,
    keys.X: note.D,
    keys.D: note.Ds,
    keys.C: note.E,
    keys.V: note.F,
    keys.G: note.Fs,
    keys.B: note.G,
    keys.H: note.Gs,
    keys.N: note.A,
    keys.J: note.As,
    keys.M: note.B,

    keys.Q: note.C5,
    50: note.Cs5,
    keys.W: note.D5,
    51: note.Ds5,
    keys.E: note.E5,
    keys.R: note.F5,
    53: note.Fs5,
    keys.T: note.G5,
    54: note.Gs5,
    keys.Y: note.A5,
    55: note.As5,
    keys.U: note.B5,

    keys.I: note.C6,
    57: note.Cs6,
    keys.O: note.D6,
    48: note.Ds6,
    keys.P: note.E6,
}

In [11]:
_keyd = {}

In [12]:
@app.event
def key_event(key, action, modifiers):
            
    keys = app.window.keys
    value = action == keys.ACTION_PRESS

    if key == keys.UP:
        state.up = value

    if key == keys.DOWN:
        state.down = value

    if key == keys.RIGHT:
        state.right = value

    if key == keys.LEFT:
        state.left = value
        
    if action == keys.ACTION_PRESS and key in keyboard:
        assert key not in _keyd
        _keyd[key] = tb303.play_new(note=keyboard[key])
        
    if action == keys.ACTION_RELEASE and key in keyboard:
        _keyd.pop(key).play_release()

In [13]:
@app.run_me_every(1/24)
def modify_oscilloscope(ct, dt):
    
    s = 2 ** dt
    
    if state.up:
        state.amp *= s
        label0.text = 'amp: %.1f' % state.amp

    if state.down:
        state.amp /= s
        label0.text = 'amp: %.1f' % state.amp

    if state.right:
        state.ms *= s
        state.ms = min(256, state.ms)
        label1.text = 'span: %.1f ms' % state.ms

    if state.left:
        state.ms /= s
        label1.text = 'span: %.1f ms' % state.ms

In [14]:
@app.event
def render(ct, dt):
    
    app.window.clear(color='#555')
    
    im, ts, te = get_oscilloscope_as_image(
        1/app.interval,
        ms=state.ms, 
        amp=state.amp, 
        color=255, 
        size=(512, 256)
    )
    
    oscilloscope.image = im    
    oscilloscope.draw()
    
    keyboard_layout.draw()
    
    label0.draw()
    label1.draw()
    label2.draw()

In [15]:
#
# Uncomment to see logging messages in case of trouble.
#
#app.get_logging_widget()

In [16]:
app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [17]:
#
# Uncomment to set the tb303 wave function to square wave.
#
#tb303.osc0.shape = 'pulse'
#

In [18]:
#
# Uncomment the following line to set up a beautiful reverb effect that 
# adds a sense of space and a touch of realism to the synthesized instrument. 
# Try it with a pair of good headphones. 
#
#set_effects(ConvolutionReverb('./sounds/impulses/MaesHowe.flac'))
#

In [19]:
#
# The reverb effect might introduce some latency to the virtual keyboard.
# If that happens you can either switch your computer's power plan to performance mode,
# or disable the reverb effect by uncommenting and running the following line:
#
#set_effects()
#

In [20]:
#
# The following line turns MIDI on. If you have an electric piano, hook it up 
# to your computer and it should just work.
#

In [21]:
app.set_midi_sound(tb303)

In [22]:
xylo = Sample('sounds/VCSL/Xylophone/Xylophone - Medium Mallets.sfz')
xylo.amp = 8

In [23]:
#
# Uncomment to set MIDI sound to a sampled Xylophone.
#
#app.set_midi_sound(xylo)
#

In [24]:
@app.sonic_live_loop(times=32)
async def loop0():
            
    use(tb303, duration=1.7, amp=0.4)

    play('C2')
    await sleep(3.)

    play('E2')
    await sleep(3.)

    play('C2')
    await sleep(6.)


In [25]:
@app.sonic_live_loop
async def loop1():
    
    use(xylo, duration=1., amp=8)
        
    play(note.C5)
    await sleep(1.)

    play(note.E5)
    await sleep(1.)

    play(note.G5)
    await sleep(1.)

In [26]:
#
# Uncomment and run to stop the two live loops.
#
#app.stop(loop0)
#app.stop(loop1)
#